In [1]:
import cobrakbase

cobrakbase 0.2.8


In [2]:
kbase = cobrakbase.KBaseAPI()

In [82]:
model1 = kbase.get_object('97644/6751/1', None)

In [83]:
model2 = kbase.get_object('82183/215/1', None)

In [86]:
info = kbase.get_object_info('97644/6751/1')
info.id

'MetBV1'

In [84]:
model1['biomasses'] = [model2['biomasses'][1]]

In [87]:
kbase.save_object('MetBV1_bio2', 97644, 'KBaseFBA.FBAModel', model1)

[[6756,
  'MetBV1_bio2',
  'KBaseFBA.FBAModel-14.0',
  '2021-08-27T22:05:03+0000',
  1,
  'filipeliu',
  97644,
  'janakakbase:narrative_1630098222612',
  'f15a48e811f3351bd974145df6024326',
  181989,
  {'Number gapgens': '0',
   'Type': 'Test',
   'Number gapfills': '1',
   'Source ID': 'GCF_000007345.1.RAST.fbamodel',
   'Number biomasses': '1',
   'Number compartments': '2',
   'Genome': '78503/3147/1',
   'Source': 'KBase',
   'Number compounds': '143',
   'Number reactions': '112',
   'Name': 'GCF_000007345.1.RAST.fbamodel'}]]

In [3]:
template = kbase.get_from_ws('CoreArchaeaTemplateMPA', 82183)

In [35]:
from cobra.core import Model
template_model = Model('model')
template_model.add_reactions([rxn.to_reaction() for rxn in template.reactions if rxn.id in t.split()])

In [34]:
t = """
rxn05467_c
rxn46184_c
rxn05759_c
rxn17445_c
rxn02480_c
rxn03079_c
rxn06299_c
rxn15961_c
rxn03085_c
rxn03020_c
rxn03127_c
rxn03126_c
rxn10471_c
rxn08173_c
"""

In [58]:
import cobra
cobra.io.save_json_model(template_model, '/home/fliu/workspace/lol.json')

In [49]:
from modelseedpy import MSBuilder
template_model.add_reactions(MSBuilder.build_exchanges(template_model))
template_model.add_reactions(MSBuilder.build_exchanges(template_model, extra_cell='c0'))
template_model.objective = 'EX_cpd01024_e0'

In [51]:
for r in template_model.reactions:
    r.lower_bound = -1000
    r.upper_bound = 1000

In [55]:
template_model.medium = {
    'EX_cpd00011_e0': 1000,
    'EX_cpd00067_e0': 1000,
    'EX_cpd00067_c0': 1000,
    'EX_cpd00001_c0': 1000,
 'EX_cpd01024_e0': 0,
 'EX_cpd00011_c0': 0,
 'EX_cpd00008_c0': 0,
 'EX_cpd00009_c0': 0,
 
 'EX_cpd00002_c0': 0,

 'EX_cpd02438_c0': 0,
 'EX_cpd02817_c0': 0,
 'EX_cpd01024_c0': 0,
 'EX_cpd02935_c0': 0,
 'EX_cpd02655_c0': 0,
 'EX_cpd00936_c0': 0,
 'EX_cpd00649_c0': 0,
 'EX_cpd11640_c0': 0,
 'EX_cpd00792_c0': 0,
 'EX_cpd02246_c0': 0,
 'EX_cpd02737_c0': 0,
 'EX_cpd00895_c0': 0,
 'EX_cpd02679_c0': 0,
 'EX_cpd08702_c0': 0,
 'EX_cpd08701_c0': 0,
 'EX_cpd27090_c0': 0,
 'EX_cpd27757_c0': 0,
 'EX_cpd27506_c0': 0,
 'EX_cpd28082_c0': 0,
 'EX_cpd11620_c0': 0,
 'EX_cpd11621_c0': 0
}

In [56]:
template_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [27]:
all_cpd = {}
for i in t.split():
    rxn = template_model.reactions.get_by_id(i + '_c0')
    for m in rxn.metabolites:
        all_cpd[m.id] = m.name

In [28]:
import pandas as pd

In [61]:
data = {}
for i in t.split():
    rxn_id = i + '0'
    data[rxn_id] = {}
    rxn = template_model.reactions.get_by_id(rxn_id)
    metabolites = dict(map(lambda x: (x[0].id, x[1]), rxn.metabolites.items()))
    for o in all_cpd:
        data[rxn_id][f'{o} {all_cpd[o]}'] = metabolites[o] if o in metabolites else 0
pd.DataFrame(data).transpose().to_csv('/home/fliu/workspace/lol.tsv', sep='\t')

In [59]:
template.reactions.rxn08173_c

Reaction identifier,rxn08173_c
Name,F(1)-ATPase
Memory address,0x07fa38adb2730
Stoichiometry,cpd00008_c + cpd00009_c + 4 cpd00067_e <=> cpd00001_c + cpd00002_c + 3 cpd00067_c ADP + Phosphate + 4 H+ <=> H2O + ATP + 3 H+
GPR,tcpx00334 or tcpx00325 or tcpx00261 or tcpx00326 or tcpx00262 or tcpx00327 or tcpx00263 or...
Lower bound,-1000
Upper bound,1000
